# CKO JAR Revision

### Import modules

In [11]:
import pandas as pd
import numpy as np
import rpy2.rinterface #ggplot tool

## Review TNIC-3 data

### Import TNIC3 data from Hoberg and Philips data library 

In [12]:
# !wget -P ../2_pipeline/ http://hobergphillips.tuck.dartmouth.edu/idata/tnic3_data.zip
# !unzip -q ../2_pipeline/tnic3_data.zip -d ../2_pipeline/ && rm ../2_pipeline/tnic3_data.zip

In [13]:
"""
Hoberg and Philips TNIC3 database
"""
tnic = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/tnic3_data.txt', 
                   delimiter='\t', header=0, index_col=['gvkey1', 'year', 'gvkey2'])
tnic.dropna(inplace=True)

/Users/ohn0000/.virtualenvs/cko-o4Ho2HSi/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Subset to firms with more than 20 competitors

In [14]:
# tnic_industry = tnic_industry.groupby(level=['gvkey1', 'year']).filter(lambda x: x.size == 20)
# tnic_industry = tnic_industry.to_frame(name='score')
# tnic_industry.to_pickle('../2_pipeline/tnic_industry.pkl')

# tnic_industry = pd.read_pickle('../2_pipeline/tnic_industry.pkl')

In [15]:
"""
tnic_industry['gvkey1'] = tnic_industry['gvkey1'].apply(lambda x: str(x).zfill(6))
tnic_industry['gvkey2'] = tnic_industry['gvkey2'].apply(lambda x: str(x).zfill(6))
"""

"\ntnic_industry['gvkey1'] = tnic_industry['gvkey1'].apply(lambda x: str(x).zfill(6))\ntnic_industry['gvkey2'] = tnic_industry['gvkey2'].apply(lambda x: str(x).zfill(6))\n"

Remeber that _year_ in __tnic_industry__ is the base year for identifying close competitors. Accordingly, _lead1_ is the M&A year and _lead2_ is the year following M&A.

Readme_tnic3.txt explains that _year_ equals the first four digits of the __compustat__ _datadate_.

### Shift years in __tnic_industry__ to get _lead1_ adn _lead2_ similarity scores

In [16]:
# tnic_industry.rename(columns={'score':'score_0'}, inplace=True)

# for i in range(1,3):
#     tnic_industry['score'] = np.NaN
#     tnic_industry.index = tnic_industry.index.set_levels(tnic_industry.index.levels[1] + 1, level=1)
#     tnic_industry.update(tnic)
#     tnic_industry.rename(columns={'score':'score' + '_' + str(i)}, inplace=True)
    
# tnic_industry.index = tnic_industry.index.set_levels(tnic_industry.index.levels[1] - 2, level=1)

In [17]:
# tnic_industry.to_pickle('../2_pipeline/tnic_industry.pkl')

### Run __*tnic_industry.py*__ on _WRDS_ to update lead scores

Many of the _lead1_ and _lead2_ values are missing. Grab these values from __TNIC_Advanced__ uploaded on _WRDS_. 

In [18]:
# !scp ../2_pipeline/tnic_industry.pkl tnic_industry.py $WRDS:~

### Download updated __*tnic_industry*__ file from WRDS

In [19]:
# !scp $WRDS:/scratch/ou/hohn/tnic_ind_update.pkl ../2_pipeline/ 

In [20]:
tnic_industry = pd.read_pickle('../2_pipeline/tnic_ind_update.pkl')

Average TNIC similarity score across 20-closest competitors.  
Remeber that in __TNIC_ALL__ most of the scores equals to zero. The _z\__ might be the more suitable.

In [21]:
avg_sim = tnic_industry.groupby(level=['gvkey1','year']).mean()
avg_sim = avg_sim.join(tnic_industry.groupby(level=['gvkey1','year']).count().add_prefix("n_"))
avg_sim = avg_sim.join(tnic_industry.fillna(0).groupby(level=['gvkey1','year']).mean().add_prefix("z_"))

In [22]:
# avg_sim.dropna() 
# # 55581 observations with non-missing scores

# avg_sim[(avg_sim['n_score_0'] == 20) & (avg_sim['n_score_1'] == 20) & (avg_sim['n_score_2'] == 20)]
# # 1483 observations with all 20 competitors present in TNIC

## Import previously constructed datasets

### Materiality of M&A

In [23]:
material = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/materiality.csv')
material.set_index(["year", "gvkey1"], inplace=True, verify_integrity=True)

Be careful since the _year_ here refers to the M&A firm-year. The _year_ in __avg_sim__ is the year competitors are identified. 

# M&A Disclosure

In [24]:
disc = pd.read_csv('/Users/ohn0000/Project/cko/0_data/manual/disc.csv', parse_dates=['DATADATE'])
disc['CIK'] = disc['CIK'].apply(lambda x: str(int(x)).zfill(10) if pd.notnull(x) else None)

Disclosure also might need additonal data collection.

In [25]:
disc.rename(columns={"GVKEY":"gvkey1", "FYEAR":"year"}, inplace=True)
disc.set_index(["year", "gvkey1"], inplace=True, verify_integrity=True)

In [26]:
manual = disc.join(material)[['DATADATE', 'CIK', 'TGTAT_ACQAT', 'TGTDVAL_ACQAT', 'MD_A', 'PROFORMA']].sort_index()

### SDC and Compustat Link File

The link file is from [Michael Ewens](https://github.com/michaelewens/SDC-to-Compustat-Mapping.git). Cite papers below.

```
@article{phillips2013r,
  title={R\&D and the Incentives from Merger and Acquisition Activity},
  author={Phillips, Gordon M and Zhdanov, Alexei},
  journal={The Review of Financial Studies},
  volume={26},
  number={1},
  pages={34--78},
  year={2013},
  publisher={Society for Financial Studies}
  }
 ```

```
@article{ewensPetersWang2018,
 title={Acquisition prices and the measurement of intangible capital},
 author={Ewens, Michael and Peters, Ryan and Wang, Sean},
 journal={Working Paper}
 year={2018}
 }
```

In [27]:
sdc_link = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/dealnum_to_gvkey.csv', 
                       dtype={'DealNumber':'Int64', 'agvkey':'Int64', 'tgvkey':'Int64'}, index_col='DealNumber')

In [28]:
import wrds
db = wrds.Connection(wrds_username = "yaera")
ma_details_desc = db.describe_table('sdc', 'ma_details').sort_values('name')
# with pd.option_context('display.max_rows', None):
#     print(ma_details_desc)

Loading library list...
Done
Approximately 5489960 rows in sdc.ma_details.


|     Variable | Description                    |
|:------------:|:-------------------------------|
|bookvalue     |Target Book Value (\$mil)       |
|compete       |Competing Bidder (Y/N)          |
|competecode   |Competing Bid Deal Code         |  
|dateann       |Date Announced                  |
|dateannest    |_dateann_ is estimated (Y/N)    | 
|dateeff       |Date Effective                  | 
|ebitltm       |Target EBIT LTM (\$mil)         |
|pct_cash      |Percentage of consideration paid in cash|
|pct_other|Percentage of consideration paid in other then cash or stock|
|pct_stk|Percentage of consideration paid in stock|
|pct_unknown|Percentage of consideration which is unknown|
|ptincltm|Target Pre-Tax Income LTM (\$mil)|
|salesltm|Target Sales LTM (\$mil)|
|rankval|Ranking Value incl Net Debt of Target (\$mil)|

Run sql query below on _WRDS_

In [29]:
# import wrds
# sdc_query = """
# select master_deal_no as DealNumber, 
#         bookvalue, 
#         compete, 
#         competecode, 
#         dateann, 
#         dateannest, 
#         dateeff, 
#         ebitltm, 
#         pct_cash,
#         pct_other,
#         pct_stk,
#         pct_unknown,
#         ptincltm,
#         salesltm,
#         rankval
# from sdc.ma_details
# where dateeff is not null 
# """
# # and master_deal_no in %(deal_no)s
# sdc = db.raw_sql(sdc_query, date_cols=['dateann', 'dateeff'])
# sdc.to_pickle('/home/upenn/yaera/sdc.pkl')

In [30]:
sdc = pd.read_pickle('/Users/ohn0000/Project/cko/0_data/external/sdc.pkl')
sdc.drop_duplicates('dealnumber', inplace = True)
sdc['dealnumber'] = sdc['dealnumber'].astype('int64')

# clear up values and change dtype to 'float'
for column in ['bookvalue', 'ebitltm', 'pct_cash', 'pct_other', 'pct_stk', 'pct_unknown', 'ptincltm', 'salesltm', 'rankval']:
    sdc[column] = sdc[column].apply(lambda x: np.NaN if x == '*********' else (np.NaN if pd.isna(x) else (float(x.replace(',', '')) if isinstance(x, str) else float(x))))
    sdc[column].astype('float16')

In [31]:
sdc['dateeff'].max()

Timestamp('2019-10-18 00:00:00')

In [32]:
sdc_sub = pd.merge(sdc_link, sdc, left_index=True, right_on='dealnumber').drop('dealnumber', axis='columns')
sdc_sub.index.name = 'dealnumber'

In [33]:
sdc_sub.sort_values(['agvkey', 'dateeff'], inplace=True)

Use __compustat__ _datadate_ and gvkey to link the sdc data to the similarity scores

In [40]:
import wrds
db = wrds.Connection(wrds_username = 'hohn')

sdc_quary = """
select gvkey, datadate, fyear, cusip,  cik
from comp.funda
where consol = %(consol)s and indfmt in %(indfmt)s and datafmt = %(datafmt)s and popsrc = %(popsrc)s and curcd in %(curcd)s
"""

parm = {'consol':('C'), 'indfmt' : ('INDL', 'FS'), 'datafmt': ('STD'), 'popsrc' : ('D'), 'curcd' : ('USD', 'CAD')}

funda = db.raw_sql(sdc_quary, params = parm, date_cols = ['datadate'])

Loading library list...
Done


In [41]:
funda['start'] = funda['datadate'] - pd.DateOffset(months = 12) + pd.DateOffset(days = 1)
funda['gvkey'] = funda['gvkey'].astype('int64')
funda.set_index('gvkey', inplace=True)

In [42]:
funda.fyear = funda.fyear.astype('Int16')

In [43]:
import pandasql as ps

sql_query = '''
select a.*, b.datadate, b.fyear, b.cusip, b.cik
from sdc_sub a left join funda b
on a.agvkey = b.gvkey and a.dateeff between b.start and b.datadate
'''

newdf = ps.sqldf(sql_query, locals())

In [44]:
col = list(newdf)
for i in range(2, 6):
    col.insert(i, col.pop(-1))
newdf = newdf.loc[:,col]

In [45]:
for i in ['datadate', 'dateann', 'dateeff']:
    newdf[i] = newdf[i].astype('datetime64[ns]')
    
newdf['year'] = newdf['datadate'].dt.year.astype('Int16')
for i in ['fyear', 'agvkey', 'tgvkey']:
    newdf[i] = newdf[i].astype('Int64')

In [46]:
col = list(newdf)
col.insert(col.index('datadate'), col.pop(col.index('year')))
newdf = newdf.loc[:,col]

In [47]:
newdf = newdf.drop_duplicates(subset='dealnumber')

In [48]:
newdf[newdf['agvkey'].notnull()]

,dealnumber,agvkey,cik,cusip,fyear,year,datadate,tgvkey,bookvalue,compete,...,dateannest,dateeff,ebitltm,pct_cash,pct_other,pct_stk,pct_unknown,ptincltm,salesltm,rankval
0,2238597,1004,0000001750,000361105,1997,1998,1998-05-31,NaN,NaN,None,...,No,1997-06-19,NaN,NaN,NaN,NaN,NaN,NaN,45.000,NaN
1,2273624,1004,0000001750,000361105,1997,1998,1998-05-31,NaN,NaN,None,...,No,1997-10-24,NaN,NaN,NaN,NaN,NaN,NaN,18.000,NaN
2,2570557,1004,0000001750,000361105,2000,2001,2001-05-31,1300,NaN,None,...,No,2000-09-29,NaN,18.75,81.25,NaN,NaN,NaN,20.000,0.016
3,3307499,1004,0000001750,000361105,2006,2007,2007-05-31,NaN,NaN,None,...,No,2007-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3419980,1004,0000001750,000361105,2007,2008,2008-05-31,NaN,NaN,None,...,No,2007-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33059,1029741,289735,0001565228,92836Y201,2016,2016,2016-12-31,NaN,NaN,None,...,No,2016-02-01,NaN,NaN,100.00,NaN,NaN,NaN,12.238,3.000
33060,402724,296753,0001528903,N07831105,2013,2013,2013-12-31,NaN,NaN,None,...,No,2013-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33061,850888,314866,0001492966,G6583A102,2015,2015,2015-08-31,NaN,NaN,No,...,No,2015-06-25,NaN,100.00,NaN,NaN,NaN,NaN,NaN,559.000
33062,687462,315318,0001590714,28618M106,2014,2014,2014-12-31,NaN,NaN,None,...,No,2014-10-01,NaN,NaN,NaN,NaN,100.0,NaN,169.854,401.553


In [49]:
newdf['rankval'].count()

18994

18994 observations with non-missing _rankval_

In [50]:
newdf['salesltm'].count()

8055

8055 observations with non-missing _salesltm_

In [51]:
np.sum(newdf['rankval'].notnull() & newdf['salesltm'].notnull())

6445

6445 observations with both _rankval_ and _salesltm_ available

## Append similarity score between acquirer and target

In [69]:
upload = newdf[newdf['agvkey'].notnull() & newdf['tgvkey'].notnull() & newdf['year'].notnull()][['agvkey', 'tgvkey', 'year']].rename(columns={'agvkey':'gvkey1', 'tgvkey':'gvkey2'})
upload.to_csv('/Users/ohn0000/Project/cko/2_pipeline/upload.csv', index=False)
!scp /Users/ohn0000/Project/cko/2_pipeline/upload.csv $WRDS:/scratch/ou/hohn

upload.csv                                    100%  113KB 448.0KB/s   00:00    


Run this on wrds server. The __TNIC_All__ files should be uploaded in scratch beforehand.

In [ ]:
"""
The server killed the previous code that joins after combines all files. The current code instead loop over the files.
"""
# !cd /scratch/ou/hohn/TNIC_AllPairsDistrib
# !cat tnicall1996.txt > tnicall_combined.txt
# !for file in tnicall{1997..2017}.txt; do sed '1d' $file >> tnicall_combined.txt; done
# !cd ~


"""
atsim.py
"""


In [72]:
!scp atsim.py $WRDS:~

atsim.py                                      100%  939    19.4KB/s   00:00    


In [ ]:
!scp $WRDS:/scratch/ou/hohn/atsim.csv /Users/ohn0000/Project/cko/2_pipeline/

In [ ]:
col = list(newdf)
col.insert(col.index('bookvalue'), col.pop(col.index('atsim')))
newdf = newdf.loc[:,col]

## IV candidates

The materiality measure based on deal value will be the last resort for the IV.   
Alternatively, 2SLS using multiple IVs is feasible.

Candidates
* Max deal value
* Sum deal value
* Datedif between _dateeff_ and _datadate_
    * _dateeff_ of the first M&A
    * _dateeff_ of the largest M&A
    * weighted average of _dateeff_ 

## Cross-sections
* Similarity between acquirer and target 
    - Relation stronger in diversifying
    - Could be more of a U-shaped relation, i.e., competitors don't follow when you move far enough
* Average value of pre-similarities between acquirer and close competitors 
    - Prediction not clear
* M&A performance during the completed firm-year
    - Relation stronger when M&A was more successful <-> how do we define success of an M&A?
* Number of close competitors of the target
    - Potential targets are candidates of future mergers
* How many competitors were there initially?
    - The size of the TNIC industry